In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# İmport Section
import matplotlib.pyplot as plt
import seaborn as sns

# Information
The COVID-19 pandemic in India is a part of the worldwide pandemic of coronavirus disease 2019 (COVID-19) caused by severe acute respiratory syndrome coronavirus 2 (SARS-CoV-2). The first case of COVID-19 in India, which originated from China, was reported on 30 January 2020. Currently, India has the largest number of confirmed cases in Asia. As of 12 June 2021, India has the second-highest number of confirmed cases in the world (after the United States) with 29.3 million reported cases of COVID-19 infection and the third-highest number of COVID-19 deaths (after the United States and Brazil) at 367,081 deaths.<br>
<br>
<br>
<br>
**Data columns**<br>
State/UTs : Names of Indian States and Union Territories.<br>
Total Cases : Total number of confirmed cases<br>
Active : Total number of active cases<br>
Discharged : Total number of discharged cases<br>
Deaths : Total number of deaths<br>
Active Ratio : Ratio of active cases to total cases<br>
Discharge Ratio : Ratio of discharged cases to total cases<br>
Death Ratio : Ratio of deaths to total cases<br>

## Load and Check Data

In [ ]:
df=pd.read_csv("../input/latest-covid19-india-statewise-data/Latest Covid-19 India Status.csv")
df.info() # as we can see there is no missing value

In [ ]:
df.shape

In [ ]:
df.describe()


In [ ]:
df.head()

# # Data Analysis
    -

In [ ]:
color_list=["lime"]
pd.plotting.scatter_matrix(df,
                          figsize=[15,15],
                          c=color_list,
                          diagonal='hist',
                          alpha=0.5,
                          s=200,
                          marker='.',
                          edgecolor='black') 
plt.show()

# Correlation 


In [ ]:
sns.heatmap(df.corr(),annot=True,fmt='.2f')

## We can see that there is correlation between <br>
Total Cases -(Active,Discharged,Deats)<br>
Active -(Discharged,Deaths)<br>
Discharged-Deaths<br>
<br>

Active Ratio-Discharge Ratio

In [ ]:
#now visualization
def total_other(fea):
    for i in fea:
        sns.regplot(x=df["Total Cases"],y=df[i])
        plt.xticks(rotation=90)
        plt.show()
total_other(["Active","Discharged","Deaths"])

In [ ]:
def total_other(fea):
    for i in fea:
        sns.regplot(x=df["Active"],y=df[i])
        plt.xticks(rotation=90)
        plt.show()
total_other(["Discharged","Deaths"])

In [ ]:

 sns.regplot(x=df["Discharged"],y=df["Deaths"])
 plt.xticks(rotation=90)
 plt.show()


## Most Deaths 

In [ ]:
df_=df.sort_values(by='Deaths',ascending=False).iloc[0:5]
df_

In [ ]:
sns.barplot(x=df_['State/UTs'],y=df_.Deaths)
plt.show()

## Active case 

In [ ]:
df_=df.sort_values(by="Active",ascending=False).iloc[:5]
df_

In [ ]:
sns.barplot(x=df_["State/UTs"],y=df_.Active)

## Prediction part
Discharged-Active<br>
Deaths-Active<br>
Discharged- Deaths<br>
Total Cases -(Active,Discharged,Deats)<br>


## Discharged-Active

In [ ]:
#Discharged-Active
sns.scatterplot(y=df.Discharged,x=df.Active,marker="*",s=400,color="purple")

from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

y=df.Discharged.values.reshape(-1,1)
x=df.Active.values.reshape(-1,1)

linear_regression=LinearRegression()
linear_regression.fit(x,y)
y_head=linear_regression.predict(x)
# for reliability
print("r2_score : " ,r2_score(y,y_head)) #0.75688871935345 our confidence rate is 75 not good


plt.plot(x,y_head)

In [ ]:
#Decision Tree ile
from sklearn.tree import DecisionTreeRegressor
tree_reg=DecisionTreeRegressor()
y=df.Discharged.values.reshape(-1,1)
x=df.Active.values.reshape(-1,1)
tree_reg.fit(x,y)
x_=np.arange(min(x),max(x),100).reshape(-1,1)
y_head=tree_reg.predict(x_)
plt.ylabel("Discharged")
plt.xlabel("Active")
plt.scatter(x,y)
plt.plot(x_,y_head)
 


## Deaths-Active

In [ ]:
y=df.Deaths.values.reshape(-1,1)
x=df.Active.values.reshape(-1,1)
plt.xlabel("Active")
plt.ylabel("Deaths")
plt.scatter(x,y)
#Again with decision tree
tree_reg.fit(x,y)
x_=np.arange(min(x),max(x),100).reshape(-1,1)
y_head=tree_reg.predict(x_)
plt.plot(x_,y_head)

## Discharged- Deaths

In [ ]:
y=df.Deaths.values.reshape(-1,1)
x=df.Discharged.values.reshape(-1,1)
plt.scatter(x,y) #ı think there is a linear relation

linear_regression=LinearRegression()
linear_regression.fit(x,y)
y_head=linear_regression.predict(x)
# for reliability
print("r2_score : " ,r2_score(y,y_head)) #0.8465507400257989 our confidence rate is 84 not bad
plt.xlabel("Discharged")
plt.ylabel("Deaths")

plt.plot(x,y_head)

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
pol_reg=PolynomialFeatures(degree=4)

x_pol=pol_reg.fit_transform(x)
linear_regression.fit(x_pol,y)
y_head=linear_regression.predict(x_pol)

plt.scatter(x,y)
plt.xlabel("Discharged")
plt.ylabel("Deaths")
plt.plot(x_pol,y_head)

print("r2_score : " ,r2_score(y,y_head)) # 0.9588840996715609 much better

## Total Cases- Active

In [ ]:

x=df["Active"].values.reshape(-1,1)
y=df["Total Cases"].values.reshape(-1,1)
plt.scatter(x,y)
plt.xlabel("Active")
plt.ylabel("Total Cases")
#with decision tree
tree_reg.fit(x,y)
x_=np.arange(min(x),max(x),100).reshape(-1,1)
y_head=tree_reg.predict(x_)
plt.plot(x_,y_head)

## Total Cases- Discharged

In [ ]:
x=df["Discharged"].values.reshape(-1,1)
y=df["Total Cases"].values.reshape(-1,1)
plt.scatter(x,y)
plt.xlabel("Discharged")
plt.ylabel("Total Cases")
#linear regression
linear_regression=LinearRegression()
linear_regression.fit(x,y)
y_head=linear_regression.predict(x)
plt.plot(x,y_head)
# for reliability
print("r2_score : " ,r2_score(y,y_head)) #0.999 already true


## Total Cases- Deaths

In [ ]:
y=df["Deaths"].values.reshape(-1,1)
x=df["Total Cases"].values.reshape(-1,1)
plt.scatter(x,y)
plt.ylabel("Deaths")
plt.xlabel("Total Cases")

#ı think its polynomial
pol_reg=PolynomialFeatures(degree=3)
x_pol=pol_reg.fit_transform(x)
linear_regression.fit(x_pol,y)
y_head=linear_regression.predict(x_pol)
plt.plot(x_pol,y_head)

print("r2_score : " ,r2_score(y,y_head)) # 0.9567761376968864